In [1]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
from scipy.special import expit


In [2]:
def abrirImagenesEscaladas( carpeta, escala=300 ):
    # abre todas las imagenes de la carpeta, y las escala de tal forma que midan (escala x escala)px
    # devuelve las imagenes aplanadas -> vectores de tamano escala^2 con valores entre 0 y 1
    imagenes = []

    for dirpath, dirnames, filenames in os.walk(carpeta):
        for file in filenames:
            img = Image.open( os.path.join(carpeta, file) )
            img = img.resize((escala, escala))
            img.convert('1')
            img = np.asarray(img)
            if len(img.shape)==3:
                img = img[:,:,0].reshape((escala**2 )) / 255
            else:
                img = img.reshape((escala**2 )) / 255
            
            imagenes.append( img )

    return imagenes

In [3]:
def balancear_datos(imagenes_entrenamiento):
    imagenes_entrenamiento_balanceadas = abrirImagenesEscaladas(imagenes_entrenamiento)
    return imagenes_entrenamiento_balanceadas

In [4]:
im=np.array(balancear_datos("train_mini2"))
x_t = im
y_t = np.array([0]*42 + [1]*42)
TOLERANCIA = 0.0001


In [5]:
def normalizar(X):
     return (X - np.mean(X, axis=0)) / np.std(X, axis=0)

In [6]:
def f(x,w,b):
    return (np.tanh((np.dot(w, x.T)) + b) + 1) / 2

In [7]:
def L(w, b, x, y):
    f = (np.tanh(np.dot(x, w) + b) + 1) / 2
    return np.mean((f - y) ** 2)

In [8]:
def descxgrad(X, y_true, learning_rate, iter=1000):
    k = X.shape[1]  # número de píxeles por imagen
    n = X.shape[0]  # número de imágenes
    X = normalizar(X)  # Normalizar los datos

    w = np.ones(k)  # Inicializar w con valores pequeños aleatorios
    b = 0

    for i in range(iter):
        y_predicha = f(X,w,b)
        error = y_predicha - y_true
        w_grad = np.dot(error * (1 - y_predicha**2), X) / n
        b_grad = np.sum(error * (1 - y_predicha**2)) / n

        w_sig = w - learning_rate * w_grad
        b_sig = b - learning_rate * b_grad

        diferencia = abs(L(w,b,X,y_true)- L(w_sig,b_sig,X,y_true))
        if diferencia<TOLERANCIA:
            print(f'iteracion {i}, min: {L(w_sig,b_sig,X,y_true)}')
            break

        w=w_sig
        b=b_sig
        print(f'iteracion {i}, min: {L(w,b,X,y_true)}')
    return w,b, y_predicha

In [9]:
alfas = [30,29,28,27,26,25,24,23,22,21,20,15,10,5,2,1,0.5,0.1,0.05,0.001]
for i in range(len(alfas)):
    print(f'alfa: {alfas[i]}')
    descxgrad(x_t,y_t,alfas[i])
    print('\n')

alfa: 30
iteracion 0, min: 0.3333333333333333
iteracion 1, min: 0.08333333333333333
iteracion 2, min: 0.09523809523809523
iteracion 3, min: 0.047619047619047616
iteracion 4, min: 0.047619047619047616


alfa: 29
iteracion 0, min: 0.3333333333333333
iteracion 1, min: 0.08333333333333333
iteracion 2, min: 0.09523809523809612
iteracion 3, min: 0.047619047619047616
iteracion 4, min: 0.047619047619047616


alfa: 28
iteracion 0, min: 0.32142857142857145
iteracion 1, min: 0.08333333333333333
iteracion 2, min: 0.08333333333333333


alfa: 27
iteracion 0, min: 0.32142857142857145
iteracion 1, min: 0.08333333333333333
iteracion 2, min: 0.09523809523809523
iteracion 3, min: 0.05952380952380952
iteracion 4, min: 0.03571428571428571
iteracion 5, min: 0.03571428571428571


alfa: 26
iteracion 0, min: 0.32142857142857145
iteracion 1, min: 0.07142857142857142
iteracion 2, min: 0.047619047619047616
iteracion 3, min: 0.047619047619047616


alfa: 25
iteracion 0, min: 0.32142857142857145
iteracion 1, min: 0.

In [10]:
rate = 25
w,b,y_mono =  descxgrad(x_t,y_t,rate)

iteracion 0, min: 0.32142857142857145
iteracion 1, min: 0.07142857142857142
iteracion 2, min: 0.08333333333333333
iteracion 3, min: 0.07142857142857142
iteracion 4, min: 0.047619047619047616
iteracion 5, min: 0.03571428571428571
iteracion 6, min: 0.03571428571428571


In [11]:
prediccion = f(x_t,w,b)
prediccion

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])